<a href="https://colab.research.google.com/github/vidyaana22/portofolio-dataanalyst/blob/main/Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
with zipfile.ZipFile("/content/uts-nlp-teknologi-sains-data-ftmm-2023.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import pandas as pd

In [ ]:
#Membaca dataset train
dftrain = pd.read_csv("traindata.csv")
dftrain.head()

,Id,Sentiment,Pasangan Calon,Text Tweet
0,1,negative,Agus-Sylvi,Banyak akun kloning seolah2 pendukung #agussil...
1,2,negative,Agus-Sylvi,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,3,negative,Agus-Sylvi,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,4,negative,Agus-Sylvi,Kasian oh kasian dengan peluru 1milyar untuk t...
4,5,negative,Agus-Sylvi,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...


In [ ]:
#Membaca dataset test
dftest = pd.read_csv("testdata.csv")
dftest.head()

,Id,Text Tweet
0,807,Pasangan #AniesSandi merupakan pasangan yang i...
1,189,Sinyal Positif Agus-Sylvi Merapat ke Anies-San...
2,223,Keren #AHY
3,408,"dulu #Jakarta tidak pernah terpecah belah, tap..."
4,8,Terima Kasih teruntuk pendukung #AHY #SYLVI ka...


In [ ]:
for i in range(10):
  print(dftrain['Text Tweet'][i])

Banyak akun kloning seolah2 pendukung #agussilvy mulai menyerang paslon #aniessandi dengan opini dan argumen pmbenaran..jangan terkecoh
#agussilvy bicara apa kasihan yaa...lap itu air matanya wkwkwkwk
Kalau aku sih gak nunggu hasil akhir QC tp lagi nunggu motif cuitan pak @SBYudhoyono kayak apa.. pasca #AgusSilvy Nyungsep..
Kasian oh kasian dengan peluru 1milyar untuk tiap RW #agussilvy tidak mempan menangin pilkada #QuickCount #PilkadaSerentak2017
Maaf ya pendukung #AgusSilvy..hayo dukung #AniesSandi diputaran 2 @ronavioleta @NetizenTofa
Kepada Allah SWT kami ucapkan rasa syukur dan kepada pak @SBYudhoyono @presidenSBY #AHY kami sampaikan terimakasih. Makna 17% suara anda.
Trima Kasih atas KeIstiqomahan Relawan #AHY @AgusYudhoyono lah yg tlh mengantarkan #AniesSandiDKI1 #JakartaGubernurBaru dg cukup signifikan
Mengenang pidato kekalahan #ahy
Batal nyoblos, baru tau ternyata ga ada no.1 di kertas suara. #AHY #kangen #mosing	
sudah boleh Ngakak? survey mu jauh panggang dari api ! #Ahy


In [ ]:
dftrain['Sentiment'].value_counts()

positive    381
negative    369
Name: Sentiment, dtype: int64

# **Preprocessing**

In [ ]:
import re
from bs4 import BeautifulSoup
import unicodedata

In [ ]:
def umum(text):
    text = text.strip().lower()
    text = unicodedata.normalize('NFKD', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r"\d+", "", text)
    return text

#Elemen Non-ASCII
def hapus_nonascii(text):
    return ''.join(char for char in text if ord(char) < 128)

#Menghapus Emoticon
def hapus_emot(text):
    emoticon_ranges = [
        '\U0001F600-\U0001F64F', '\U0001F300-\U0001F5FF', '\U0001F680-\U0001F6FF',
        '\U0001F700-\U0001F77F', '\U0001F780-\U0001F7FF', '\U0001F800-\U0001F8FF',
        '\U0001F900-\U0001F9FF', '\U0001FA00-\U0001FA6F', '\U0001FA70-\U0001FAFF',
        '\U0001FB00-\U0001FBFF', '\U0001FC00-\U0001FCFF', '\U0001FD00-\U0001FDFF',
        '\U0001FE00-\U0001FEFF', '\U0001FF00-\U0001FFFF']

    emoticon_pattern = r'[' + '|'.join(emoticon_ranges) + ']+'
    return re.sub(emoticon_pattern, '', text)

#Data dari Twitter
def twitter(text):
    text = re.sub(r'#\w+', '', text)   # Hapus hashtags
    text = re.sub(r'@\w+', '', text)   # Hapus mentions
    return text

#Menghapus html
def html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Hapus kurung siku
def hapuskurung(text):
    return re.sub('\[[^]]*\]', '', text)

#Preprocessing terakhir
def prep_final(text, remove_digits=False):
    text = text.replace("\n", " ")

    # Hapus simbol
    if remove_digits:
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    else:
        text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

    text = re.sub(' +', ' ', text)
    return text

#Fungsi prepro yang diperlukan disatukan
def clean(text):
    text = umum(text)
    text = hapus_nonascii(text)
    text = hapus_emot(text)
    text = twitter(text)
    text = prep_final(text)
    return text

#Apply function on Text Tweet column
dftrain_basic = dftrain.copy()
dftrain_basic['basic_prep'] = dftrain_basic['Text Tweet'].apply(clean)

In [ ]:
pd.concat([dftrain['Text Tweet'], dftrain_basic['basic_prep']], axis=1)

,Text Tweet,basic_prep
0,Banyak akun kloning seolah2 pendukung #agussil...,banyak akun kloning seolah pendukung mulai men...
1,#agussilvy bicara apa kasihan yaa...lap itu ai...,bicara apa kasihan yaa lap itu air matanya wk...
2,Kalau aku sih gak nunggu hasil akhir QC tp lag...,kalau aku sih gak nunggu hasil akhir qc tp lag...
3,Kasian oh kasian dengan peluru 1milyar untuk t...,kasian oh kasian dengan peluru milyar untuk ti...
4,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...,maaf ya pendukung hayo dukung diputaran
...,...,...
745,"Ingat, pilih no.3 di tanggal 19 April nanti ya...",ingat pilih no di tanggal april nanti ya kakak...
746,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ...",kali saja bpk lihat lagi di rspun tetap selfie...
747,Kita harus dapat merangkul semua orang tanpa b...,kita harus dapat merangkul semua orang tanpa b...
748,Ini jagoanku dibidang digital <Smiling Face Wi...,ini jagoanku dibidang digital smiling face wit...


In [ ]:
dftrain['Text Tweet'] = dftrain['Text Tweet'].apply(clean)
dftest['Text Tweet'] = dftest['Text Tweet'].apply(clean)

# **Stopwords Remove**

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

list_stopwords = list(set(stopwords.words('indonesian')))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
list_stopwords.extend(['yg', 'dgn', 'tp', 'krn', 'bgt', 'jg', 'dtg', 'blm', 'udh',
                      'sdh', 'sm', 'bs', 'kl', 'kalo', 'br', 'lbh', 'pgn', 'pengen',
                      'msh', 'skrg', 'sp', 'jgn', 'yaa', 'lgi', 'sll', 'sbg', 'pd',
                      'utk', 'km', 'sgt', 'stlh', 'sblm', 'ttg', 'spt'])
list_stopwords.remove('tidak')
print(len(list_stopwords))

790


In [ ]:
def stopwords_remove(text):
    tokens = word_tokenize(text)
    filtered_tokens = [t for t in tokens if t not in list_stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#Apply function on Text Tweet column
dftrain_stopwords = dftrain.copy()
dftrain_stopwords['stopwords_removal'] = dftrain_stopwords['Text Tweet'].apply(stopwords_remove)

In [ ]:
pd.concat([dftrain['Text Tweet'], dftrain_stopwords['stopwords_removal']], axis=1)

,Text Tweet,stopwords_removal
0,banyak akun kloning seolah pendukung mulai men...,akun kloning pendukung menyerang paslon opini ...
1,bicara apa kasihan yaa lap itu air matanya wk...,bicara kasihan lap air matanya wkwkwkwk
2,kalau aku sih gak nunggu hasil akhir qc tp lag...,sih gak nunggu hasil qc nunggu motif cuitan ka...
3,kasian oh kasian dengan peluru milyar untuk ti...,kasian oh kasian peluru milyar rw tidak mempan...
4,maaf ya pendukung hayo dukung diputaran,maaf ya pendukung hayo dukung diputaran
...,...,...
745,ingat pilih no di tanggal april nanti ya kakak...,pilih no tanggal april ya kakak ok hand
746,kali saja bpk lihat lagi di rspun tetap selfie...,kali bpk lihat rspun selfie ok hand
747,kita harus dapat merangkul semua orang tanpa b...,merangkul orang batas usia kelamin ok hand vic...
748,ini jagoanku dibidang digital smiling face wit...,jagoanku dibidang digital smiling face with su...


In [ ]:
dftrain['Text Tweet'] = dftrain['Text Tweet'].apply(stopwords_remove)
dftest['Text Tweet'] = dftest['Text Tweet'].apply(stopwords_remove)

# **Stemming**

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
from nltk import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer = StemmerFactory().create_stemmer()

def sastrawi_stem(text):
    tokens = word_tokenize(text)
    tokens_stemmed = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens_stemmed)

#Apply function on Text Tweet column
dftrain_stem = dftrain.copy()
dftrain_stem['stemmed'] = dftrain_stem['Text Tweet'].progress_apply(sastrawi_stem)

100%|██████████| 750/750 [02:26<00:00,  5.14it/s]


In [ ]:
pd.concat([dftrain['Text Tweet'], dftrain_stem['stemmed']], axis=1)

,Text Tweet,stemmed
0,akun kloning pendukung menyerang paslon opini ...,akun kloning dukung serang paslon opini argume...
1,bicara kasihan lap air matanya wkwkwkwk,bicara kasihan lap air mata wkwkwkwk
2,sih gak nunggu hasil qc nunggu motif cuitan ka...,sih gak nunggu hasil qc nunggu motif cuit kaya...
3,kasian oh kasian peluru milyar rw tidak mempan...,kasi oh kasi peluru milyar rw tidak mempan men...
4,maaf ya pendukung hayo dukung diputaran,maaf ya dukung hayo dukung putar
...,...,...
745,pilih no tanggal april ya kakak ok hand,pilih no tanggal april ya kakak ok hand
746,kali bpk lihat rspun selfie ok hand,kali bpk lihat rspun selfie ok hand
747,merangkul orang batas usia kelamin ok hand vic...,rangkul orang batas usia kelamin ok hand victo...
748,jagoanku dibidang digital smiling face with su...,jago bidang digital smiling face with sunglass...


In [ ]:
dftrain['Text Tweet'] = dftrain['Text Tweet'].apply(sastrawi_stem)
dftest['Text Tweet'] = dftest['Text Tweet'].apply(sastrawi_stem)

# **TF-IDF**

In [ ]:
X = dftrain["Text Tweet"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Tfidf vectorizer
tfidf = TfidfVectorizer()
tfidf_x = tfidf.fit_transform(X)

x_test = tfidf.transform(dftest['Text Tweet'])
print('Tfidf_train:', tfidf_x.shape)
print('Tfidf_test:',x_test.shape)

Tfidf_train: (750, 1924)
Tfidf_test: (150, 1924)


In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

y_train = lb.fit_transform(dftrain['Sentiment'])
print(y_train[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


# **Build Model**

## Logistic Regression 0.77

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)

tfidf_logreg = logreg.fit(tfidf_x,y_train)
print(tfidf_logreg)

LogisticRegression(C=1, max_iter=500, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#Predict model
predict_logreg = logreg.predict(x_test)
print(predict_logreg)

[1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0
 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1
 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 1
 0 0]


In [ ]:
predicted = pd.DataFrame()
predicted['Category'] = pd.Series(predict_logreg)

def transform_label(label):
    return 'positive' if label == 1 else 'negative'

predicted['Category'] = predicted['Category'].apply(transform_label)

output = pd.concat([dftest['Id'], predicted['Category']], axis=1)
output

In [ ]:
output = pd.concat([dftest['Id'], predicted['Category']], axis=1)
output

,Id,Category
0,807,positive
1,189,positive
2,223,positive
3,408,negative
4,8,positive
...,...,...
145,44,positive
146,747,negative
147,541,positive
148,203,negative


In [ ]:
output.to_csv('Vidyana_162112133002.csv', index=False)

## Random Forest 0.77

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=300)
tfidf_clf = clf.fit(tfidf_x, y_train)
print(tfidf_clf)

<ipython-input-70-8632d034b7d1>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  tfidf_clf = clf.fit(tfidf_x, y_train)


RandomForestClassifier(n_estimators=300)


In [ ]:
#Predict model
predict_clf = logreg.predict(x_test)
print(predict_clf)

[1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0
 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1
 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 1
 0 0]


In [ ]:
predicted = pd.DataFrame()
predicted['Category'] = pd.Series(predict_clf)

def transform_label(label):
    return 'positive' if label == 1 else 'negative'

predicted['Category'] = predicted['Category'].apply(transform_label)

output = pd.concat([dftest['Id'], predicted['Category']], axis=1)
output

,Id,Category
0,807,positive
1,189,positive
2,223,positive
3,408,negative
4,8,positive
...,...,...
145,44,positive
146,747,negative
147,541,positive
148,203,negative


In [ ]:
output.to_csv('submission2.csv', index=False)